# Using AutoGen with motleycrew

Microsoft AutoGen is one of the most popular multi-agent frameworks. Motleycrew supports interaction with autogen in both directions: firstly, you can wrap an AutoGen chat as a motleycrew tool, which you can then give to any supported agent; secondly, you can give any motleycrew tool (which includes agents used as tools) to an AutoGen agent. 

The two options are described in detail below

Let's install external dependencies for the example.

In [1]:
%pip install duckduckgo-search
%pip install ag2


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## Integrating an AutoGen chat into motleycrew

If you want to use an AutoGen chat (say one from their many wonderful examples, or one you have already) as part of a motleycrew setup, you can wrap it as a tool, which you can then give to any other motleycrew-supported agent.

Let's create an AutoGen chat for automatic problem solving.
The code is taken from the example here: https://microsoft.github.io/autogen/docs/notebooks/agentchat_groupchat

In [3]:
import autogen
import os

llm_config = {
    "config_list": [{"model": "gpt-4.1", "api_key": os.environ["OPENAI_API_KEY"]}],
    "cache_seed": None,
}

In [4]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "examples/data/groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    max_consecutive_auto_reply=1,  # Limit auto-replies to prevent infinite loops
    human_input_mode="NEVER",
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config,
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

If we were using plain AutoGen, we'd start the chat with something like this:
```
user_proxy.initiate_chat(
    manager, message="Find a latest paper about gpt-4 on arxiv and find its potential applications in software."
)
```

You see, the chat accepts an input and returns an output, just like a tool.  
Because of that, for using the chat in motleycrew, we can utilize the built-in AutoGenChatTool. Its prompt can either be a template or a plain string. Here we are creating a tool that searches arXiv for recent papers.

In [5]:
from motleycrew.tools.autogen_chat_tool import AutoGenChatTool

knowledge_retrieval_tool = AutoGenChatTool(
    name="retrieve_knowledge_by_topic",
    description="Search arxiv for the latest paper on a given topic "
                "and find its potential applications in software.",  # will be used in the prompt of the future agent that will use the tool
    prompt="Find a latest paper about {topic} on arxiv "
            "and find its potential applications in software.",  # this is the initial prompt for the AutoGen chat itself
    initiator=user_proxy,
    recipient=manager,
)

/Users/whimo/motleycrew/.venv/lib/python3.11/site-packages/lunary/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


We can now give the tool to any agent and solve tasks with it.

In [6]:
from motleycrew import MotleyCrew
from motleycrew.agents.langchain import ReActToolCallingMotleyAgent

crew = MotleyCrew()
writer = ReActToolCallingMotleyAgent(tools=[knowledge_retrieval_tool])

In [7]:
from motleycrew.tasks import SimpleTask

blog_post_task = SimpleTask(
        crew = crew,
        name="Produce blog post on the applications of latest advancements related to GPT-4",
        description="Using the insights provided by searching research papers, develop an engaging blog "
                    "post that highlights the most significant advancements on GPT-4 ant their applications.\n"
                    "Your post should be informative yet accessible, catering to a tech-savvy audience.\n"
                    "Make it sound cool, avoid complex words so it doesn't sound like AI. "
                    "Create a blog post of at least 4 paragraphs.",
        agent=writer,
    )

In [8]:
crew.run()

User_proxy (to chat_manager):

Find a latest paper about GPT-4 advancements and applications on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------

Next speaker: Coder

Coder (to chat_manager):

Plan:
1. Use Python to search arXiv for the latest paper about "GPT-4 advancements and applications".
2. Display the paper title, abstract, and arXiv link.
3. Use the abstract to analyze and suggest potential software applications myself.

First, let's execute the Python code to fetch and present info about the latest relevant paper from arXiv.

```python
import requests
import xml.etree.ElementTree as ET

# Search query parameters
search_query = "GPT-4 advancements applications"
url = (f"http://export.arxiv.org/api/query?search_query=all:{search_query}"
       "&start=0&max_results=1&sortBy=submittedDate&sortOrder=descending")

response = requests.get(url)
root = ET.fromstring(response.content)

# Extract tit

[TaskUnit(status=done)]

In [9]:
from IPython.display import display, Markdown
display(Markdown(blog_post_task.output))

Final Answer: 

**Unveiling the Future: GPT-4's Groundbreaking Advancements and Their Real-World Applications**

In the ever-evolving landscape of artificial intelligence, GPT-4 stands as a beacon of innovation, pushing the boundaries of what machines can achieve. Recent research has shed light on some of the most exciting advancements in GPT-4, particularly in the realm of auditory processing. Imagine a world where AI can understand and interact with sound as seamlessly as humans do. This is the frontier that GPT-4 is exploring, and the implications are nothing short of revolutionary.

One of the standout findings from recent studies is the introduction of an "auditory Turing test," designed to benchmark GPT-4's audio capabilities against human performance. While humans excel with a 52% accuracy in complex auditory tasks, GPT-4 and other state-of-the-art models currently lag behind at 6.9%. This gap highlights the challenges AI faces in replicating human-like auditory scene analysis, such as selective attention and noise robustness. However, this isn't just a limitation—it's an opportunity. By identifying these gaps, researchers are paving the way for AI systems that can better understand and interact with the world through sound.

The potential applications of these advancements are vast and varied. For instance, security software can leverage complex audio-based challenges for human verification, making it harder for automated systems to bypass security measures. In the realm of accessibility, audio-first products like smart speakers and hearing aids can benefit from improved AI diagnostics, ensuring they perform reliably in real-world environments. Moreover, transcription tools and meeting assistants can use these insights to enhance their performance in noisy or multi-speaker settings, making them more effective and user-friendly.

As we look to the future, the integration of these advancements into software products promises to enhance not only the functionality but also the reliability of AI systems. From audio forensic analysis to next-gen multimodal systems that combine vision and sound, the possibilities are endless. By adopting the latest research frameworks, developers can ensure their AI-powered products are not only cutting-edge but also aligned with human capabilities. In essence, GPT-4 is not just a tool—it's a gateway to a more intuitive and interactive digital world.

The writer agent used the AutoGen chat tool to retrieve data for the post!



## Using any motleycrew-supported tools with Autogen agents

Now let's do it the other way around: give tools (or agents, for that matter) from motleycrew to AutoGen agents. The code is based on https://microsoft.github.io/autogen/docs/tutorial/tool-use

In [10]:
from motleycrew.tools import MotleyTool, RetryConfig
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = MotleyTool.from_supported_tool(
    DuckDuckGoSearchRun(),
    retry_config=RetryConfig(max_retries=5)  # for retrying rate limit errors
)  # Any tools or even motleycrew's agents can be converted to MotleyTool like this!

# Let's first define the assistant agent that suggests tool calls.
assistant = autogen.ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can provide useful and up-to-date data using web search. "
    "After providing a complete answer, always respond with 'TERMINATE' to end the conversation.",
    llm_config=llm_config,
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = autogen.ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,  # Limit auto-replies to prevent infinite loops
)

# Make an autogen-compatible tool from the MotleyTool.
autogen_tool = search_tool.to_autogen_tool()

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="search_tool", description="Web search tool")(autogen_tool)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="search_tool")(autogen_tool)

In [11]:
chat_result = user_proxy.initiate_chat(
    assistant,
    message="What was the first computer? Search the web for the answer.",
    max_turns=3  # Limit total conversation turns to prevent infinite loops
)

User (to Assistant):

What was the first computer? Search the web for the answer.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_l03oEy2Oce4O363k3yunzyeF): search_tool *****
Arguments: 
{"input":"What was the first computer in history?"}
****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search_tool...
Call ID: call_l03oEy2Oce4O363k3yunzyeF
Input arguments: {'input': 'What was the first computer in history?'}


/Users/whimo/motleycrew/.venv/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



>>>>>>>> EXECUTED FUNCTION search_tool...
Call ID: call_l03oEy2Oce4O363k3yunzyeF
Input arguments: {'input': 'What was the first computer in history?'}
Output:
Jun 16, 2025 · Until this breakthrough, all the mechanical aids to calculation were merely calculators or, like the Difference Engine, glorified calculators. The Analytical Engine, although …Computer - History, Technology, Innovation: A computer might be described with deceptive simplicity as “an apparatus that performs routine calculati… Jan 5, 2025 · The Atanasoff–Berry Computer (ABC): Created by John Vincent Atanasoff and Clifford Berry in the 1930s, the ABC was the world's first electronic digital computer. It used … Jun 14, 2025 · In 1822, Charles Babbage conceptualized and began developing the Difference Engine, the first automatic computing machine to approximate polynomials. The Difference … Apr 4, 2025 · The Universal Automatic Computer I (UNIVAC I), developed by Eckert and Mauchly, was the first commercially successful